# Notes
    1. For volume, one feature may be the # of trajectories in window
    2. Try average pooling
    3. Fill missing value
    4. Scaling?
    5. Training date?
    6. multi-dim cnn
    7. remove 10-1
    8.custom loss = MAPE
    9.valid am 227 pm 223
    
# Records
    Best 9 weeks
    

# CNN dev




In [1]:
from src import functions as func
from src import feagen as feag
import datetime
import numpy as np
from datetime import datetime,timedelta
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Dense,Activation
from keras.layers.core import Flatten,Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn import preprocessing
import keras

def my_mape(label,pred):
    c = 0.0
    mae = 0.0
    for (l,p) in zip(label.flatten(),pred.flatten()):
        if l != 0:
            mae += abs(l-p)/l
            c += 1
    return mae/c


win_6 = (6-6)*3
win_8 = (8-6)*3
win_10 = (10-6)*3
win_15 = (15-6)*3
win_17 = (17-6)*3
win_19 = (19-6)*3
task = 'am'
if task == 'am':
    x_range = range(win_6,win_8)
    y_range = range(win_8,win_10)
else:
    x_range = range(win_15,win_17)
    y_range = range(win_17,win_19)

Using TensorFlow backend.


## Feature Engineering

In [2]:
#Read training data
path = '../dataSets/training/'
file_suffix = '.csv'
in_file = 'trajectories(table 5)_training'
travel_times = func.read_file_to_travel_times(path, in_file, file_suffix)


#Read testing data
path = '../dataSets/testing_phase1/'
file_suffix = '.csv'
in_file = 'trajectories(table 5)_test1'
testing_times = func.read_file_to_travel_times(path, in_file, file_suffix)

#merge them into one dict
routes =  travel_times.keys()
for r in routes:
    travel_times[r].update(testing_times[r])


In [19]:
total_weeks = 6 # ?(train) + 1(valid) + 1(test), take off 10/1~7
training_weeks = total_weeks-2
testing_weeks = total_weeks-1
num_instance = total_weeks*7 # 13 weeks
num_routes = 6
num_window = 13*3 # 6 ~ 19
routes = travel_times.keys()
print routes
window_size = timedelta(minutes=20)
window_start = datetime.strptime('2016-09-13 06:00:00', "%Y-%m-%d %H:%M:%S")
window_end = datetime.strptime('2016-09-13 19:00:00', "%Y-%m-%d %H:%M:%S")


cube_2d = np.zeros((num_instance*num_window,num_routes))

d=0
while d < num_instance:
    window_scanner = window_start
    for t in range(num_window):
        for r in range(num_routes):
            try:
                cube_2d[d*39+t][r] = np.mean(travel_times[routes[r]][window_scanner])
            except:
                print 'No ',str(window_start.date())
                cube_2d[d*39+t][r] = 0
        window_scanner += window_size
    window_start += timedelta(days=1)
    d += 1

print cube_2d.shape
                       


['B-3', 'B-1', 'A-3', 'A-2', 'C-3', 'C-1']
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-13
No  2016-09-14
No  2016-09-14
No  2016-09-14
No  2016-09-14
No  2016-09-14
No  2016-09-14
No  2016-09-14
No  2016-09-14
No  2016-09-14
No  2016-09-14
No  2016-09-14
No  2016-09-14
No  2016-09-14
No  2016-09-14
No  2016-09-14
No  2016-09-14
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-15
No  2016-09-

# AM task

In [20]:
# Normalize
normed = preprocessing.normalize(cube_2d,axis=0)
#normed = preprocessing.MinMaxScaler().fit_transform(cube_2d)
cube_3d = np.zeros((num_instance,num_window,num_routes))
for d in range(num_instance):
    for t in range(num_window):
        for r in range(num_routes):
            cube_3d[d][t][r] = cube_2d[d*39+t][r] if t in y_range else normed[d*39+t][r]

    
trX = cube_3d[:training_weeks*7,x_range,:]
trY = cube_3d[:training_weeks*7,y_range,:]
valX = cube_3d[training_weeks*7:testing_weeks*7,x_range,:]
valY = cube_3d[training_weeks*7:testing_weeks*7,y_range,:]
ttX = cube_3d[testing_weeks*7:,x_range,:]


print 'Train data:',trX.shape
print 'Valid data:',valX.shape
print 'Test data:',ttX.shape
print 'Valid non-zero',np.count_nonzero(valY.flatten())
print 'Test non-zero:',np.count_nonzero(ttX)



Train data: (28, 6, 6)
Valid data: (7, 6, 6)
Test data: (7, 6, 6)
Valid non-zero 227
Test non-zero: 201


In [21]:
print trX[0]
print trX[1]

[[ 0.05810657  0.13700536  0.18550417  0.15724254  0.          0.        ]
 [ 0.06633221  0.12604609  0.20904696  0.25521947  0.19907982  0.36979237]
 [ 0.07866735  0.10720785  0.26065949  0.18721891  0.          0.        ]
 [ 0.08190291  0.14932656  0.26750391  0.18171638  0.          0.27054241]
 [ 0.07831378  0.15916151  0.30248475  0.2358807   0.28716626  0.18925479]
 [ 0.08102786  0.12914667  0.1908613   0.24664791  0.          0.39898278]]
[[ 0.          0.          0.18546608  0.15868092  0.          0.        ]
 [ 0.08980166  1.          0.23458973  0.23472425  0.          0.30823113]
 [ 0.06837362  0.14759661  0.19123431  0.22773213  0.          0.22053758]
 [ 0.09227836  0.11700512  0.19361526  0.25357907  0.          0.        ]
 [ 0.06252794  0.          0.26951819  0.17765672  0.28801673  0.37042331]
 [ 0.04212893  0.18741452  0.          0.21914838  0.          0.38869467]]


In [22]:
epoch =4000
epoch_display = 100
step = epoch/epoch_display


filter_lenghts = [2,3,6]
filter_strides = [2,3,1]
nb_filters = 12
kernel_size = [3,3]
out_dim = 3*6

'''
conv_filters = []
travel_time = Input(shape=(trX.shape[1], trX.shape[2]))
for i,j in zip(filter_lenghts,filter_strides):
    conv = Conv1D(nb_filters, i,strides = j)(travel_time)
    act = Activation('sigmoid')(conv)
    conv_filters.append(act)
merged_vector = keras.layers.concatenate(conv_filters, axis=1)
#droped_merged_vector = Dropout(0.5)(merged_vector)
#pooled = keras.layers.pooling.MaxPooling1D(pool_size=2,strides=2)(merged_vector)

layer1 = Dense(12,activation='linear')(merged_vector)
predictions = Dense(6)(layer1)

model = Model(inputs=travel_time, outputs=predictions)

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.00001)
model.compile(loss='mae',optimizer=adam)


best = 0.3
for i in range(step):
    model.fit(trX, trY, epochs=epoch_display, verbose=0)
    predY = model.predict(valX)
    mape = my_mape(valY,predY)
    best = mape if best > mape else best
    print 'epoch :',epoch_display*(i+1),' Val :',mape,'\n'
print 'Best mape :',best

'''
am_model = Sequential()
am_model.add(Conv1D(12, 1,input_shape=(trX.shape[1], num_routes)))


#am_model.add(Dropout(0.5))
am_model.add(Dense(8,activation='relu'))
#am_model.add(Activation('sigmoid'))
am_model.add(Dense(6,activation='linear'))
#am_model.add(Activation('linear'))


adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
am_model.compile(loss='mse',optimizer=adam)

best = 0.3
for i in range(step):
    am_model.fit(trX, trY, epochs=epoch_display, verbose=0)
    predY = am_model.predict(valX)
    mape = my_mape(valY,predY)
    if best > mape :
        best = mape 
    print 'epoch :',epoch_display*(i+1),' Val :',mape,'\n'
print 'Best mape :',best



epoch : 100  Val : 0.23502827025 

epoch : 200  Val : 0.207875415339 

epoch : 300  Val : 0.202135409828 

epoch : 400  Val : 0.199050800145 

epoch : 500  Val : 0.197414714706 

epoch : 600  Val : 0.196296751578 

epoch : 700  Val : 0.195551711727 

epoch : 800  Val : 0.194990731132 

epoch : 900  Val : 0.194510386319 

epoch : 1000  Val : 0.194106083023 

epoch : 1100  Val : 0.193778860621 

epoch : 1200  Val : 0.193523985511 

epoch : 1300  Val : 0.19332939466 

epoch : 1400  Val : 0.193193376512 

epoch : 1500  Val : 0.193104191503 

epoch : 1600  Val : 0.193043798598 

epoch : 1700  Val : 0.193005213783 

epoch : 1800  Val : 0.192982788836 

epoch : 1900  Val : 0.192972235996 

epoch : 2000  Val : 0.192970537224 

epoch : 2100  Val : 0.192975874225 

epoch : 2200  Val : 0.192987568569 

epoch : 2300  Val : 0.193006149652 

epoch : 2400  Val : 0.19303357582 

epoch : 2500  Val : 0.193074232124 

epoch : 2600  Val : 0.193138485479 

epoch : 2700  Val : 0.19326248104 

epoch : 2800  

In [23]:
print predY[1,:,0]
print valY[1,:,0]

print my_mape(valY[1,:,0],predY[1,:,0])
predY = am_model.predict(ttX)
print ttX[0,:,0]
print predY[0,:,0]

[ 103.86331177  113.0643692   106.65476227  113.1995697   109.89897919
  117.1073761 ]
[  84.05        156.475       207.035       117.824       168.475
  156.67333333]
0.272912998105
[ 0.03652508  0.08073544  0.06273641  0.08045525  0.04896696  0.05997785]
[ 101.3215332   114.25783539  123.85884857  113.40641022  114.77292633
  123.9340744 ]


In [24]:
print (0.185574931909*227+0.203402560198*223)/(227+223)

0.19440951215


In [27]:

print '2016-10-03'>'2016-10-02'

True
